In [1]:
wd = '/mnt/openfact/users/msawinski/factue-task2'
import sys, os
os.chdir(wd)

In [2]:
import pandas as pd
from pathlib import Path
import os
root = Path("data/llm_output/persuasion")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)

In [3]:
df['source_file'].value_counts().sort_index()

source_file
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-bg/batch_0003.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-bg/batch_0008.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-bg/batch_0009.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-bg/batch_0010.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-bg/batch_0012.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-bg/batch_0015.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-bg/batch_0016.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justific

In [4]:
df.head(3)

,filename,start,end,text_lang,text,label_bin,labels_multi,identifier,technique_id,raw_result,Description,Verdict,pred,gold,source_file
0,RU_discrediting_the_west_1.txt,1635,1834,RU,Человека превратят в суррогат.\nА суррогату не...,True,[Consequential_Oversimplification],ModelName.LLAMA_31_8B-justification/Appeal_to_...,Appeal_to_Values,"[{\n ""Description"": ""Appeal to Values techn...",[Appeal to Values technique was used by refere...,"[True, True, True, True, True]",True,False,data/llm_output/persuasion/detect/ModelName.LL...
1,RU_discrediting_the_west_1.txt,1836,2138,RU,"Нам очень важно спасти людей от этого, уберечь...",True,[Appeal_to_Values],ModelName.LLAMA_31_8B-justification/Appeal_to_...,Appeal_to_Values,"[{\n ""Description"": ""Appeal to Values techniq...",[Appeal to Values technique was used by evokin...,"[True, True, True, True, True]",True,True,data/llm_output/persuasion/detect/ModelName.LL...
2,RU_discrediting_the_west_1.txt,2140,2176,RU,Человек должен оставаться человеком.,True,[Conversation_Killer],ModelName.LLAMA_31_8B-justification/Appeal_to_...,Appeal_to_Values,"[{\n ""Description"": ""This phrase appeals to t...",[This phrase appeals to the value of humanity ...,"[True, True, True, False, True]",True,False,data/llm_output/persuasion/detect/ModelName.LL...


In [7]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
# Define a function to compute metrics for a group
# Assume df has columns: technique_id, text_lang, gold, pred
rows = []
for technique_id in df['technique_id'].unique():
    for text_lang in df.loc[df['technique_id'] == technique_id, 'text_lang'].unique():
        group = df[(df['technique_id'] == technique_id) & (df['text_lang'] == text_lang)]
        
        row = {
            'technique_id': technique_id,
            'text_lang': text_lang,
            'accuracy': accuracy_score(group['gold'], group['pred']),
            'recall': recall_score(group['gold'], group['pred'], zero_division=0),
            'f1': f1_score(group['gold'], group['pred'], zero_division=0),
            'support': len(group)  # ✅ count of examples
        }
        rows.append(row)

results = pd.DataFrame(rows)
print(results)

       technique_id text_lang  accuracy    recall        f1  support
0  Appeal_to_Values        RU  0.580000  0.857143  0.363636      100
1  Appeal_to_Values        BG  0.621429  1.000000  0.036364      140
2  Appeal_to_Values        PL  0.707143  1.000000  0.549451      140
3  Appeal_to_Values        SI  0.500000  0.500000  0.500000        8


# total

In [6]:


# Metrics
acc = accuracy_score(df['gold'], df['pred'])
recall = recall_score(df['gold'], df['pred'], average='binary')
f1 = f1_score(df['gold'], df['pred'], average='binary')

print(f"Accuracy: {acc:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")

Accuracy: 0.639
Recall: 0.909
F1-score: 0.364
